# Problem description

There are ten different digits and corresponding ranks from 1 to 10. Training a neural network to return the serial number of digit that has the max value of the sum of three contiguous digits.

In [1]:
import random


def get_training_set():
    size = 10000
    file = open('data/problem1_training_set.txt', 'w')
    for i in range(size):
        array = []
        number = 0
        while number < 10:
            temp = random.randint(0, 100)
            if temp not in array:
                array.append(temp)
                file.write(str(temp) + '\t')
                number += 1
#         print(array)
        array_3sum = []
        for j in range(8):
            array_3sum.append(array[j] + array[j + 1] + array[j + 2])
#         print(array_3sum)
        serial_number = array_3sum.index(max(array_3sum))
        file.write(str(serial_number) + '\n')
#         print(serial_number)


def read_training_set():
    file = open('data/problem1_training_set.txt', 'r')
    training_set = file.readlines()
    # print(training_set)
    data = []
    label = []
    for i in range(len(training_set)):
        numbers = []
        j = 0
        while j < len(training_set[i]):
            num = ''
            symbol = training_set[i][j]
            while '0' <= symbol <= '9':
                num += symbol
                j += 1
                if j < len(training_set[i]):
                    symbol = training_set[i][j]
                else:
                    break
            j += 1
            if num != '':
                numbers.append(int(num))
#         print(numbers)
        data.append(numbers[0:10])
        label.append(numbers[10])
    # print(data)
    # print(label)
    return data, label

get_training_set()
data, label = read_training_set()
print(len(data))

10000


In [2]:
size = len(data)
print(size)
divide = int(0.8 * size)
print(divide)
training_data = data[0: divide]
training_label = label[0: divide]
test_data = data[divide: size]
test_label = label[divide: size]

10000
8000


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(1, 1, (1, 3))
#         self.pool = nn.MaxPool2d((1, 8))
    
    def forward1(self, x):
        x = self.conv(x)
        print(x)
#         x = self.pool(x)
        x = torch.argmax(x)
        return x
    
    def forward2(self, x):
        x = self.conv(x)
        x = F.softmax(x, 3)
        return x
    
    def forward(self, x):
        x = self.conv(x)
        return x


net = Net()
print(net)
input_oral = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
print(input_oral)
input_tensor = torch.tensor(input_oral).float()
print(input_tensor)
print(input_tensor.size())
input_tensor = input_tensor.unsqueeze(0)
input_tensor = input_tensor.unsqueeze(0)
input_tensor = input_tensor.unsqueeze(0)
print(input_tensor.size())
print(net.forward1(input_tensor))
print(net.forward2(input_tensor))
print(torch.squeeze(net.forward(input_tensor)))

Net(
  (conv): Conv2d(1, 1, kernel_size=(1, 3), stride=(1, 1))
)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])
torch.Size([10])
torch.Size([1, 1, 1, 10])
tensor([[[[1.1029, 1.0303, 0.9577, 0.8851, 0.8125, 0.7399, 0.6673, 0.5947]]]],
       grad_fn=<MkldnnConvolutionBackward>)
tensor(0)
tensor([[[[0.1590, 0.1478, 0.1375, 0.1278, 0.1189, 0.1106, 0.1028, 0.0956]]]],
       grad_fn=<SoftmaxBackward>)
tensor([1.1029, 1.0303, 0.9577, 0.8851, 0.8125, 0.7399, 0.6673, 0.5947],
       grad_fn=<SqueezeBackward0>)


In [4]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss(reduction='sum')
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
print(torch.tensor([[1,2,3], [2,3,4]]))
print(torch.tensor([1, 2]))
print(criterion(torch.tensor([[1,2,3], [2,3,4]]).float(), torch.tensor([1, 2])))

tensor([[1, 2, 3],
        [2, 3, 4]])
tensor([1, 2])
tensor(1.8152)


In [5]:
device = torch.device("cuda: 1")

for i in range(0, len(training_data), 10):
    running_loss = 0.0
    training_part_data = torch.tensor(training_data[i: i+10]).float()
    training_part_label = torch.tensor(training_label[i: i+10])
#     print('i: ', i)   
    # mini-batch=10
    j = 0
    
#     net.to(device)
#     training_part_data.to(device)
#     training_part_label.to(device)
    
    for training_part in training_part_data:
#         training_part.cuda()
        training_part = training_part.unsqueeze(0).unsqueeze(0).unsqueeze(0)         
        training_part = torch.squeeze(net.forward(training_part)).unsqueeze(0)
        if j == 0:
            mini_batch_training = training_part
        else:
            mini_batch_training = torch.cat((mini_batch_training, training_part), dim=0)    
        j += 1
#     print('mini_batch_training: ', mini_batch_training)
#     print('training_part_label: ', training_part_label)
    
    loss = criterion(mini_batch_training, training_part_label)
    loss.backward()
    optimizer.step()
    
    # print loss for every 200 samples
    running_loss += loss.item()
    if (i+10) % 200 == 0:
        print('[%d] loss: %.3f' % (i+10, running_loss / 200))
        running_loss = 0.0
print('Finished Training')

[200] loss: 8.620
[400] loss: 80.158
[600] loss: 64.287
[800] loss: 192.427
[1000] loss: 34.293
[1200] loss: 166.107
[1400] loss: 227.011
[1600] loss: 196.556
[1800] loss: 57.992
[2000] loss: 1021.757
[2200] loss: 1626.310
[2400] loss: 872.695
[2600] loss: 682.624
[2800] loss: 71.860
[3000] loss: 0.000
[3200] loss: 0.000
[3400] loss: 2130.206
[3600] loss: 722.695
[3800] loss: 2282.527
[4000] loss: 240.056
[4200] loss: 552.693
[4400] loss: 638.043
[4600] loss: 17.878
[4800] loss: 0.000
[5000] loss: 959.884
[5200] loss: 1378.283
[5400] loss: 1108.260
[5600] loss: 951.549
[5800] loss: 690.543
[6000] loss: 899.135
[6200] loss: 655.900
[6400] loss: 0.000
[6600] loss: 27.091
[6800] loss: 0.000
[7000] loss: 892.985
[7200] loss: 350.879
[7400] loss: 592.293
[7600] loss: 1684.366
[7800] loss: 1033.265
[8000] loss: 1210.588
Finished Training


In [6]:
# net = net.cpu()
correct = 0
for i in range(len(test_data)):
    predict = torch.argmax(net.forward(torch.tensor(test_data[i]).float().unsqueeze(0).unsqueeze(0).unsqueeze(0))).item()  
    if predict == test_label[i]:
          correct+= 1

print("accuracy: %.3f" % float(correct/len(test_data)))

accuracy: 0.854


In [7]:
params = list(net.parameters())
print(params)

[Parameter containing:
tensor([[[[31478.1035, 27751.2598, 22866.7422]]]], requires_grad=True), Parameter containing:
tensor([0.5264], requires_grad=True)]
